# 0.0: Loading initial packages: 

In [41]:
## helpful packages

# Administrative stuff:
import os

# For Data manipulation:
import pandas as pd
import numpy as np
import random
import re

# Nice to haves
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
# import googlemaps
import plotly.graph_objects as go

## dynamic text updating
from IPython.display import Markdown as md

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 0.1: Initial Data Load - 2019 Data for Air and Health

In [42]:
air_data = pd.read_csv("annual_aqi_by_county_2019.csv",low_memory=False)
air_data.shape
air_data.head()

(1020, 18)

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10
0,Alabama,Baldwin,2019,271,237,34,0,0,0,0,80,52,37,0,0,220,51,0
1,Alabama,Clay,2019,107,97,10,0,0,0,0,67,50,30,0,0,0,107,0
2,Alabama,Colbert,2019,263,252,11,0,0,0,0,61,47,37,0,0,228,35,0
3,Alabama,DeKalb,2019,361,324,37,0,0,0,0,90,51,39,0,0,331,30,0
4,Alabama,Elmore,2019,228,208,20,0,0,0,0,100,50,39,0,0,228,0,0


In [43]:
# Loading initial data:
health_data = pd.read_csv("PLACES__Local_Data_for_Better_Health__Census_Tract_Data_2021_release.csv",low_memory=False)
health_data.shape
health_data.head()

(2125561, 23)

,Year,StateAbbr,StateDesc,CountyName,CountyFIPS,LocationName,DataSource,Category,Measure,Data_Value_Unit,...,Data_Value_Footnote,Low_Confidence_Limit,High_Confidence_Limit,TotalPopulation,Geolocation,LocationID,CategoryID,MeasureId,DataValueTypeID,Short_Question_Text
0,2018,AL,Alabama,Baldwin,1003,1003011601,BRFSS,Prevention,Cervical cancer screening among adult women ag...,%,...,NaN,83.5,87.1,"6,062",POINT (-87.55879555 30.41466842),1003011601,PREVENT,CERVICAL,CrdPrv,Cervical Cancer Screening
1,2019,ME,Maine,Waldo,23027,23027045000,BRFSS,Health Status,Fair or poor self-rated health status among ad...,%,...,NaN,17.4,20.3,"5,969",POINT (-69.23270168 44.42005508),23027045000,HLTHSTAT,GHLTH,CrdPrv,General Health
2,2019,AL,Alabama,Calhoun,1015,1015002300,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,...,NaN,38.0,40.0,"3,843",POINT (-85.67101785 33.93457433),1015002300,HLTHOUT,OBESITY,CrdPrv,Obesity
3,2019,LA,Louisiana,Tangipahoa,22105,22105953500,BRFSS,Health Risk Behaviors,Current smoking among adults aged >=18 years,%,...,NaN,22.3,25.4,"7,225",POINT (-90.36823496 30.77718215),22105953500,RISKBEH,CSMOKING,CrdPrv,Current Smoking
4,2019,ME,Maine,Cumberland,23005,23005014000,BRFSS,Health Risk Behaviors,Current smoking among adults aged >=18 years,%,...,NaN,14.7,20.3,"3,872",POINT (-70.61427542 43.97054122),23005014000,RISKBEH,CSMOKING,CrdPrv,Current Smoking


## Fitler for Kentucky and for the year 2019

In [44]:
health_ken = health_data[health_data["StateDesc"] == "Kentucky"].reset_index()

health_ken_2019 = health_ken[health_ken["Year"]==2019].copy()
print("Length of original health data: " + str(len(health_data.index)))
print("Length of Kentucky 2019 health data: " + str(len(health_ken_2019.index)))
health_ken.head()
health_ken.shape

Length of original health data: 2125561
Length of Kentucky 2019 health data: 24332


,index,Year,StateAbbr,StateDesc,CountyName,CountyFIPS,LocationName,DataSource,Category,Measure,...,Data_Value_Footnote,Low_Confidence_Limit,High_Confidence_Limit,TotalPopulation,Geolocation,LocationID,CategoryID,MeasureId,DataValueTypeID,Short_Question_Text
0,3596,2019,KY,Kentucky,Adair,21001,21001970300,BRFSS,Health Risk Behaviors,No leisure-time physical activity among adults...,...,NaN,41.5,48.3,"3,016",POINT (-85.15821669 37.13121962),21001970300,RISKBEH,LPA,CrdPrv,Physical Inactivity
1,3597,2019,KY,Kentucky,Adair,21001,21001970402,BRFSS,Health Outcomes,Stroke among adults aged >=18 years,...,NaN,3.0,3.6,"4,261",POINT (-85.26802881 37.12055109),21001970402,HLTHOUT,STROKE,CrdPrv,Stroke
2,3598,2019,KY,Kentucky,Anderson,21005,21005950201,BRFSS,Health Outcomes,Depression among adults aged >=18 years,...,NaN,25.5,27.9,"5,372",POINT (-84.92306901 38.02049683),21005950201,HLTHOUT,DEPRESSION,CrdPrv,Depression
3,3599,2019,KY,Kentucky,Bell,21013,21013960200,BRFSS,Health Outcomes,Stroke among adults aged >=18 years,...,NaN,5.4,6.4,"5,150",POINT (-83.73909778 36.76433928),21013960200,HLTHOUT,STROKE,CrdPrv,Stroke
4,3600,2019,KY,Kentucky,Bell,21013,21013960300,BRFSS,Health Outcomes,Stroke among adults aged >=18 years,...,NaN,5.0,6.2,"2,381",POINT (-83.61861461 36.71356868),21013960300,HLTHOUT,STROKE,CrdPrv,Stroke


(33170, 24)

In [45]:
air_ken = air_data[air_data["State"] == "Kentucky"].reset_index()

air_ken_2019 = air_ken[air_ken["Year"]==2019].copy()

print("Length of original air data: " + str(len(air_data.index)))
print("Length of Kentucky 2019 air data: " + str(len(air_ken_2019.index)))

air_ken.head()



Length of original air data: 1020
Length of Kentucky 2019 air data: 27


,index,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10
0,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0
1,327,Kentucky,Boone,2019,239,222,17,0,0,0,0,80,49,39,0,0,239,0,0
2,328,Kentucky,Boyd,2019,365,313,52,0,0,0,0,74,53,39,0,11,192,160,2
3,329,Kentucky,Bullitt,2019,243,214,29,0,0,0,0,93,51,41,0,0,243,0,0
4,330,Kentucky,Campbell,2019,362,298,63,1,0,0,0,108,56,40,0,21,189,152,0


## Number of County in both the dataset

In [46]:
#checking number of County's is both the datasets

print("Number of unique county in health data: " + str(health_ken_2019['CountyName'].nunique()))

print("Number of unique county in air data: " + str(air_ken_2019['County'].nunique()))


pd.DataFrame(health_ken_2019['CountyName'].value_counts())


pd.DataFrame(air_ken_2019['County'].value_counts())


Number of unique county in health data: 120
Number of unique county in air data: 27


,CountyName
Jefferson,4180
Fayette,1804
Kenton,902
Campbell,550
Warren,528
...,...
Gallatin,44
Powell,44
Trimble,44
Hickman,22


,County
Bell,1
Jefferson,1
Warren,1
Trigg,1
Simpson,1
Pulaski,1
Pike,1
Perry,1
Oldham,1
Morgan,1


## Number of Categories in Health Data

In [47]:
#health_ken['Category'].nunique()

print("Number of unique category in health data: " + str(health_ken_2019['Category'].nunique()))

pd.DataFrame(health_ken_2019['Category'].value_counts())

Number of unique category in health data: 4


,Category
Health Outcomes,13272
Prevention,4424
Health Risk Behaviors,3318
Health Status,3318


## Number of Measure in Health Data


In [48]:
#health_ken['Measure'].nunique()

print("Number of unique Measure in health data: " + str(health_ken_2019['Measure'].nunique()))


pd.DataFrame(health_ken_2019['Measure'].value_counts())

Number of unique Measure in health data: 22


,Measure
No leisure-time physical activity among adults aged >=18 years,1106
Stroke among adults aged >=18 years,1106
Chronic obstructive pulmonary disease among adults aged >=18 years,1106
Chronic kidney disease among adults aged >=18 years,1106
Diagnosed diabetes among adults aged >=18 years,1106
Visits to doctor for routine checkup within the past year among adults aged >=18 years,1106
Current smoking among adults aged >=18 years,1106
Current lack of health insurance among adults aged 18-64 years,1106
Binge drinking among adults aged >=18 years,1106
High blood pressure among adults aged >=18 years,1106


## Number of category id 

In [49]:
#health_ken['CategoryID'].nunique()

print("Number of unique Category Id in health data: " + str(health_ken_2019['CategoryID'].nunique()))

pd.DataFrame(health_ken_2019['CategoryID'].value_counts())

Number of unique Category Id in health data: 4


,CategoryID
HLTHOUT,13272
PREVENT,4424
RISKBEH,3318
HLTHSTAT,3318


## Number of measure id

In [50]:
#health_ken_2019['MeasureId'].nunique()

print("Number of unique Measure Id in health data: " + str(health_ken_2019['MeasureId'].nunique()))

pd.DataFrame(health_ken_2019['MeasureId'].value_counts())

Number of unique Measure Id in health data: 22


,MeasureId
LPA,1106
STROKE,1106
COPD,1106
KIDNEY,1106
DIABETES,1106
CHECKUP,1106
CSMOKING,1106
ACCESS2,1106
BINGE,1106
BPHIGH,1106


In [51]:
health_ken_2019.groupby(["Category", "Measure"]).agg('count')["CategoryID"]

Category               Measure                                                                                              
Health Outcomes        Arthritis among adults aged >=18 years                                                                   1106
                       Cancer (excluding skin cancer) among adults aged >=18 years                                              1106
                       Chronic kidney disease among adults aged >=18 years                                                      1106
                       Chronic obstructive pulmonary disease among adults aged >=18 years                                       1106
                       Coronary heart disease among adults aged >=18 years                                                      1106
                       Current asthma among adults aged >=18 years                                                              1106
                       Depression among adults aged >=18 years               

## Cleaning Health Data:

In [52]:
## drop columns that we are not gonna use
health_ken_2019_clean = health_ken_2019.drop(['StateAbbr',"LocationName", 'DataSource','StateDesc', 'CategoryID',
                                          'Data_Value_Footnote', "Data_Value_Footnote_Symbol",'DataValueTypeID', 
                                          "Short_Question_Text", "Low_Confidence_Limit", "High_Confidence_Limit"],
                                         axis=1)
health_ken_2019_clean.shape
health_ken_2019_clean.head()

(24332, 13)

,index,Year,CountyName,CountyFIPS,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,TotalPopulation,Geolocation,LocationID,MeasureId
0,3596,2019,Adair,21001,Health Risk Behaviors,No leisure-time physical activity among adults...,%,Crude prevalence,44.8,"3,016",POINT (-85.15821669 37.13121962),21001970300,LPA
1,3597,2019,Adair,21001,Health Outcomes,Stroke among adults aged >=18 years,%,Crude prevalence,3.3,"4,261",POINT (-85.26802881 37.12055109),21001970402,STROKE
2,3598,2019,Anderson,21005,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,26.6,"5,372",POINT (-84.92306901 38.02049683),21005950201,DEPRESSION
3,3599,2019,Bell,21013,Health Outcomes,Stroke among adults aged >=18 years,%,Crude prevalence,5.8,"5,150",POINT (-83.73909778 36.76433928),21013960200,STROKE
4,3600,2019,Bell,21013,Health Outcomes,Stroke among adults aged >=18 years,%,Crude prevalence,5.5,"2,381",POINT (-83.61861461 36.71356868),21013960300,STROKE


In [53]:
# ISSUE TO REVIEW 

#pivot categories column so there is one column for each health category (health outcome, health risk behaviors, prevention, and health status) 
##subset to the relvant health outcomes related to lungs (asthma, cancer, and chronic obstructive pulmonary disease) 
health_ken_2019_clean_1 = health_ken_2019_clean.pivot_table(index='Category',observed=False)
health_ken_2019_clean_1.shape
health_ken_2019_clean_1.head(3)

(4, 5)

,CountyFIPS,Data_Value,LocationID,Year,index
Category,,,,,
Health Outcomes,21110.238698,19.059803,2.111065e+10,2019,824579.768460
Health Risk Behaviors,21110.238698,24.931133,2.111065e+10,2019,827724.357444
Health Status,21110.238698,19.579325,2.111065e+10,2019,827265.377637


# 0.2: Air Quality Data: Sulfur

In [54]:
so2 = pd.read_csv("daily_42401_2019_SO2.csv")
so2_ken = so2[so2["State Name"] == "Kentucky"]

print("Length of original data: " + str(len(so2.index)))
print("Length of Kentucky data: " + str(len(so2_ken.index)))
print("Number of unique county in sulfur data: " + str(so2_ken["County Name"].nunique()))
pd.DataFrame(so2_ken["County Name"].value_counts())




Length of original data: 336542
Length of Kentucky data: 8845
Number of unique county in sulfur data: 10


,County Name
Jefferson,2179
Henderson,848
Boyd,730
Edmonson,730
Fayette,730
Jessamine,730
Campbell,728
Daviess,726
Greenup,724
McCracken,720


In [55]:
so2_ken.head()

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,...,AQI,Method Code,Method Name,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change
97694,21,19,17,42401,1,38.45934,-82.64041,WGS84,Sulfur dioxide,1 HOUR,...,0.0,100.0,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,ASHLAND PRIMARY (FIVCO),"FIVCO HEALTH DEPARTMENT, 2924 HOLT STREET",Kentucky,Boyd,Ashland,"Huntington-Ashland, WV-KY-OH",2021-11-01
97695,21,19,17,42401,1,38.45934,-82.64041,WGS84,Sulfur dioxide,1 HOUR,...,0.0,100.0,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,ASHLAND PRIMARY (FIVCO),"FIVCO HEALTH DEPARTMENT, 2924 HOLT STREET",Kentucky,Boyd,Ashland,"Huntington-Ashland, WV-KY-OH",2021-11-01
97696,21,19,17,42401,1,38.45934,-82.64041,WGS84,Sulfur dioxide,1 HOUR,...,0.0,100.0,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,ASHLAND PRIMARY (FIVCO),"FIVCO HEALTH DEPARTMENT, 2924 HOLT STREET",Kentucky,Boyd,Ashland,"Huntington-Ashland, WV-KY-OH",2021-11-01
97697,21,19,17,42401,1,38.45934,-82.64041,WGS84,Sulfur dioxide,1 HOUR,...,0.0,100.0,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,ASHLAND PRIMARY (FIVCO),"FIVCO HEALTH DEPARTMENT, 2924 HOLT STREET",Kentucky,Boyd,Ashland,"Huntington-Ashland, WV-KY-OH",2021-11-01
97698,21,19,17,42401,1,38.45934,-82.64041,WGS84,Sulfur dioxide,1 HOUR,...,0.0,100.0,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,ASHLAND PRIMARY (FIVCO),"FIVCO HEALTH DEPARTMENT, 2924 HOLT STREET",Kentucky,Boyd,Ashland,"Huntington-Ashland, WV-KY-OH",2021-11-01


In [56]:

health_county_names = list(pd.unique(health_ken_2019.CountyName))
so2_county_names = list(pd.unique(so2_ken["County Name"]))

merge_counts = [so2_names for so2_names in so2_county_names if so2_names in health_county_names]

print("Counties in SO2: " + str(len(so2_county_names)))
print("Counties in tract: " + str(len(health_county_names)))
print("Counties we can merge on: " + str(len(merge_counts)))

Counties in SO2: 10
Counties in tract: 120
Counties we can merge on: 10


## Cleaning sulfur data

pg 10 of AQI report (https://www.airnow.gov/sites/default/files/2018-04/aqi_brochure_02_14_0.pdf) 
We are using this for defining categories for sulfur aqi in categories

In [57]:
# Each county has 2 measurements: 1-hr and 3-hr but all 3-hr readings are NaN:
so2_ken = so2_ken[so2_ken["Sample Duration"] != "3-HR BLK AVG"]

# Clean and categorize AQI readings:
so2_ken_f=so2_ken.dropna(subset=["AQI"],how='all',inplace=False)
filter_method = lambda x: 'Good' if x < 50 else 'Moderate' if (x > 50 and x <= 100) else 'Unhealthy for Sensitive Groups' if (x > 100 and x <= 150) else "NA" if (x == "NaN") else "Unhealthy"
so2_ken_f["AQI_category"] = so2_ken_f["AQI"].apply(filter_method)
so2_ken_f["good_aqi"] = (so2_ken_f["AQI_category"] == "Good")
so2_ken_f["moderate_aqi"] = (so2_ken_f["AQI_category"] == "Moderate")
so2_ken_f["unhealthy_aqi"] = (so2_ken_f["AQI_category"] == "Unhealthy")
so2_ken_f["unhealthy_sens_aqi"] = (so2_ken_f["AQI_category"] == "Unhealthy for Sensitive Groups")

# Condensing data and grouping by County Name & Site Number
so2_reshape = so2_ken_f.groupby(["County Name", "Site Num"]).agg(so2_n_dates = ("Date Local", "count"),
                                                   so2_good_days = ("good_aqi", "sum"),
                                                  so2_moderate_days = ("moderate_aqi", "sum"),
                                                  so2_unhealthy_days = ("unhealthy_aqi", "sum"),
                                                                  so2_unhealthy_sens_days = ("unhealthy_sens_aqi", "sum"),
                                                                  so2_avg_aqi = ("AQI", "mean"),
                                                                  so2_max_aqi = ("AQI", "max"),
                                                                  so2_min_AQI = ("AQI", "min")).reset_index()

# Checks:
print("Length of original data: " + str(len(so2.index)))
print("Length of reshaped Kentucky data: " + str(len(so2_reshape.index)))
print("Number of unique county in original sulfur data: " + str(so2_ken["County Name"].nunique()))
print("Number of unique county in reshaped sulfur data: " + str(so2_reshape["County Name"].nunique()))

Length of original data: 336542
Length of reshaped Kentucky data: 13
Number of unique county in original sulfur data: 10
Number of unique county in reshaped sulfur data: 10


In [58]:
# Adding long lat data
so2_reshape = pd.merge(pd.DataFrame(so2_ken_f.groupby(["County Name", "Site Num", "Latitude", "Longitude", "City Name"]).size().reset_index())
                        ,so2_reshape, how = "left", on = ["County Name", "Site Num"])

so2_reshape = so2_reshape.drop([0], axis=1)
so2_reshape.rename(columns={"Latitude": "so2_lat", "Longitude": "so2_long", "City Name" : "so2_city_name"}, inplace= True)
so2_reshape

,County Name,Site Num,so2_lat,so2_long,so2_city_name,so2_n_dates,so2_good_days,so2_moderate_days,so2_unhealthy_days,so2_unhealthy_sens_days,so2_avg_aqi,so2_max_aqi,so2_min_AQI
0,Boyd,17,38.459340,-82.640410,Ashland,365,365,0,0,0,0.479452,9.0,0.0
1,Campbell,3002,39.021881,-84.474450,Highland Heights,364,364,0,0,0,1.233516,14.0,0.0
2,Daviess,5,37.780776,-87.075307,Not in a city,363,360,3,0,0,5.157025,61.0,0.0
3,Edmonson,501,37.131790,-86.142953,Not in a city,365,365,0,0,0,0.210959,3.0,0.0
4,Fayette,12,38.065030,-84.497610,Lexington-Fayette (corporate name for Lexington),365,365,0,0,0,0.676712,7.0,0.0
5,Greenup,7,38.548136,-82.731163,Worthington,362,362,0,0,0,1.317680,17.0,0.0
6,Henderson,14,37.871200,-87.463750,Not in a city,59,59,0,0,0,3.694915,17.0,0.0
7,Henderson,1011,37.654381,-87.511427,Not in a city,365,327,26,2,10,13.953425,120.0,0.0
8,Jefferson,51,38.060910,-85.898040,Louisville,360,360,0,0,0,3.175000,33.0,0.0
9,Jefferson,67,38.228760,-85.654520,Louisville,365,365,0,0,0,2.608219,26.0,0.0


In [59]:
air_ken.head()


,index,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10
0,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0
1,327,Kentucky,Boone,2019,239,222,17,0,0,0,0,80,49,39,0,0,239,0,0
2,328,Kentucky,Boyd,2019,365,313,52,0,0,0,0,74,53,39,0,11,192,160,2
3,329,Kentucky,Bullitt,2019,243,214,29,0,0,0,0,93,51,41,0,0,243,0,0
4,330,Kentucky,Campbell,2019,362,298,63,1,0,0,0,108,56,40,0,21,189,152,0


In [60]:
## Reworking health data

# create function to find unique measure IDs for each category
def unique_measures(df, category):
  df_sub = df[["Category", "Measure", "MeasureId"]]
  focus_category = df_sub[df_sub["Category"] == category]
  return focus_category.MeasureId.unique()

In [61]:
## Reworking health data

## drop columns that we are not gonna use

# print the unique measures for reference and use in the focus_measures function 
# see codebook for full measure descriptions

unique_measures(health_ken_2019_clean, category = "Health Outcomes")
# we're interested in current asthma (CASTHMA), Chronic obstructive pulmonary disease among adults aged >=18 years' (COPD)
# Cancer (excluding skin cancer) among adults aged >=18 years (CANCER) and Depression among adults aged >=18 years (DEPRESSION)

unique_measures(health_ken_2019_clean, category = "Prevention")
# we're interested in Visits to doctor for routine checkup within the past year among adults aged (CHECKUP),
# Current lack of health insurance among adults aged 18-64 years (ACCESS2), and Taking medicine for high blood pressure control among 
# adults aged >=18 years with high blood pressure' (BPMED)

unique_measures(health_ken_2019_clean, category = "Health Risk Behaviors")
# we're interested in Current smoking among adults aged >=18 years (CSMOKING) and 
# No leisure-time physical activity among adults aged >=18 years (LPA)

unique_measures(health_ken_2019_clean, category = "Health Status")
# we're interested in Physical health not good for >=14 days among adults aged >=18 years (PHLTH),
# Mental health not good for >=14 days among adults aged >=18 years (MHLTH)
# Fair or poor self-rated health status among adults aged >=18 years' (GHLTH)

array(['STROKE', 'DEPRESSION', 'ARTHRITIS', 'CASTHMA', 'HIGHCHOL', 'CHD',
       'OBESITY', 'CANCER', 'BPHIGH', 'DIABETES', 'KIDNEY', 'COPD'],
      dtype=object)

array(['CHOLSCREEN', 'BPMED', 'ACCESS2', 'CHECKUP'], dtype=object)

array(['LPA', 'BINGE', 'CSMOKING'], dtype=object)

array(['PHLTH', 'MHLTH', 'GHLTH'], dtype=object)

In [62]:
# create a function to subset to focuse measures for each category

def focus_measures(df, category, measures):
  focus_category = df[df['Category'].isin(category)]
  focus_measures = measures
  focus_category_measures = focus_category[focus_category["MeasureId"].isin(focus_measures)]
  return focus_category_measures

In [63]:
# create df subset to our focus measures 
all_categories = ["Health Outcomes", "Prevention", "Health Risk Behaviors", "Health Status"]
all_focus_measures = ["CANCER", 'COPD', 'CASTHMA', "DEPRESSION", "CHECKUP", 'ACCESS2', 'BPMED',"CSMOKING", 'LPA',"PHLTH", 'MHLTH','GHLTH']

health_ken_2019_clean.shape
health_ken_2019_fm = focus_measures(health_ken_2019_clean, category = all_categories, measures = all_focus_measures)
health_ken_2019_fm.shape

(24332, 13)

(13272, 13)

## Calculating %age of good days in Air Data

In [64]:
air_data["% of good days"] = (air_data["Good Days"] / air_data["Days with AQI"])*100

In [65]:
pd.DataFrame(air_data.groupby(["State"])["% of good days"].mean()).sort_values(["% of good days"]).reset_index()

,State,% of good days
0,Country Of Mexico,48.035964
1,District Of Columbia,66.301370
2,California,69.239879
3,Arizona,70.660467
4,Utah,76.516135
5,Illinois,77.309681
6,Delaware,77.716895
7,Georgia,78.766896
8,New Mexico,80.005232
9,New Jersey,80.113128


In [66]:
air_ken_2019["% of good days"] = (air_ken_2019["Good Days"] / air_ken_2019["Days with AQI"])*100

In [35]:
good_days_ken = pd.DataFrame(air_ken_2019.groupby(["County"])["% of good days"].mean()).sort_values(["% of good days"]).reset_index()
good_days_ken

,County,% of good days
0,Jefferson,54.794521
1,Henderson,64.406780
2,McCracken,72.022161
3,Daviess,78.021978
4,Edmonson,79.452055
5,Campbell,82.320442
6,Oldham,83.884298
7,Fayette,84.340659
8,Warren,84.770115
9,Hancock,84.897959


# 0.3: Reading Air Daily Data

In [68]:
daily_aqi = pd.read_csv("daily_aqi_by_county_2019.csv",low_memory=False)

In [69]:
daily_aqi_ken = daily_aqi[daily_aqi["State Name"]=="Kentucky"]
daily_aqi_ken.shape
daily_aqi_ken.head(2)

(8172, 10)

,State Name,county Name,State Code,County Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
105732,Kentucky,Bell,21,13,2019-01-03,26,Good,PM2.5,21-013-0002,1
105733,Kentucky,Bell,21,13,2019-01-09,8,Good,PM2.5,21-013-0002,1


## Separating by months and grouping by months 

In [70]:
daily_aqi_ken['Month'] = pd.DatetimeIndex(daily_aqi_ken['Date']).month

C:\Users\Sanha Tahir\AppData\Local\Temp\ipykernel_4300\3285874749.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_aqi_ken['Month'] = pd.DatetimeIndex(daily_aqi_ken['Date']).month


In [71]:
daily_aqi_ken['Month'].value_counts()

10    802
3     801
8     797
5     795
7     784
4     779
9     777
6     773
1     497
12    471
11    454
2     442
Name: Month, dtype: int64

# 0.4: Storing Data:

In [72]:
%store air_data
%store air_ken
%store air_ken_2019
%store health_county_names
%store health_data
%store health_ken
%store health_ken_2019
%store health_ken_2019_clean
%store health_ken_2019_fm
%store so2
%store so2_county_names
%store so2_ken
%store so2_ken_f
%store good_days_ken
%store daily_aqi
%store daily_aqi_ken

%store so2_reshape

Stored 'air_data' (DataFrame)
Stored 'air_ken' (DataFrame)
Stored 'air_ken_2019' (DataFrame)
Stored 'health_county_names' (list)
Stored 'health_data' (DataFrame)
Stored 'health_ken' (DataFrame)
Stored 'health_ken_2019' (DataFrame)
Stored 'health_ken_2019_clean' (DataFrame)
Stored 'health_ken_2019_fm' (DataFrame)
Stored 'so2' (DataFrame)
Stored 'so2_county_names' (list)
Stored 'so2_ken' (DataFrame)
Stored 'so2_ken_f' (DataFrame)
Stored 'good_days_ken' (DataFrame)
Stored 'daily_aqi' (DataFrame)
Stored 'daily_aqi_ken' (DataFrame)
Stored 'so2_reshape' (DataFrame)
